# Text Processing and Word Embeddings

Welcome to this new exercise! In this exercise, we will play around with text instead of images as before, using Recurrent Neural Networks. Generally it is called Natural Language Processing (NLP) when dealing with text, speech, etc. But the data structure is very different to images, i.e. text is string instead of numbers in images. So we need some preprocessing steps to transform raw text to other data format. And this notebook will introduce these basic concepts in NLP pipelines. Specifically, you will learn about:

1. How to preprocess text classification datasets
2. How to create a simple word embedding layer that maps words to dense vectors

# 0. Setup

As usual, we first import some packages to setup this notebook.

In [1]:
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from exercise_code.rnn.sentiment_dataset import (
    create_dummy_data,
    download_data
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# 1. Preprocessing a Text Classification Dataset

As a starting point, let's load a dummy text classification dataset and have a sense how it looks like. We take these samples from the IMDb movie review dataset, which includes movie reviews and labels that show whether they are negative (0) or positive (1). You will investigate this task further in the second notebook.

In this section, our goal is to create a text processing dataset. You are not required to write any code in this section. However, the concept introduced here is very important for working on NLP datasets in the future as well as in the rest of this exercise. So take your time to understand the procedure here. :)

First, let us download the data and take a look at some data samples.

In [2]:
i2dl_exercises_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(i2dl_exercises_path, "datasets", "SentimentData")
path = download_data(data_root)
data = create_dummy_data(path)
for text, label in data:
    print('Text: {}'.format(text))
    print('Label: {}'.format(label))
    print()

  0%|          | 0/3772493 [00:00<?, ?it/s]

3776512it [00:01, 3391740.68it/s]                             


Text: Obviously written for the stage. Lightweight but worthwhile. How can you go wrong with Ralph Richardson, Olivier and Merle Oberon.
Label: 1

Text: This is the definitive movie version of Hamlet. Branagh cuts nothing, but there are no wasted moments.
Label: 1

Text: Smallville episode Justice is the best episode of Smallville ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! It's my favorite episode of Smallville! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !
Label: 1

Text: Great movie - especially the music - Etta James -

## 1.1 Tokenizing Data

As seen above, we loaded 3 positive and 3 negative reviews. Since the basic semantic unit of text is word, the first thing we need to do is **tokenizing** the dataset, which means converting each review to a list of words.

In [3]:
import re

# use regular expression to split the sentence
# check https://docs.python.org/3/library/re.html for more information
def tokenize(text):
    return [s.lower() for s in re.split(r'\W+', text) if len(s) > 0]

tokenized_data = []
for text, label in data:
    tokenized_data.append((tokenize(text), label))
    print(tokenized_data[-1], '\n')

(['obviously', 'written', 'for', 'the', 'stage', 'lightweight', 'but', 'worthwhile', 'how', 'can', 'you', 'go', 'wrong', 'with', 'ralph', 'richardson', 'olivier', 'and', 'merle', 'oberon'], 1) 

(['this', 'is', 'the', 'definitive', 'movie', 'version', 'of', 'hamlet', 'branagh', 'cuts', 'nothing', 'but', 'there', 'are', 'no', 'wasted', 'moments'], 1) 

(['smallville', 'episode', 'justice', 'is', 'the', 'best', 'episode', 'of', 'smallville', 'it', 's', 'my', 'favorite', 'episode', 'of', 'smallville'], 1) 

(['great', 'movie', 'especially', 'the', 'music', 'etta', 'james', 'at', 'last', 'this', 'speaks', 'volumes', 'when', 'you', 'have', 'finally', 'found', 'that', 'special', 'someone'], 0) 

(['comment', 'this', 'movie', 'is', 'impossible', 'is', 'terrible', 'very', 'improbable', 'bad', 'interpretation', 'e', 'direction', 'not', 'look'], 0) 

(['a', 'rating', 'of', '1', 'does', 'not', 'begin', 'to', 'express', 'how', 'dull', 'depressing', 'and', 'relentlessly', 'bad', 'this', 'movie', 'i

In [4]:
#tokenized_data

## 1.2 Creating a Vocabulary

We have converted the dataset into pairs of token lists and corresponding labels. But strings have variant length and are less easy for handling. It would be nice to represent words with numbers. So, we need to create a <b>vocabulary</b>, which is a dictionary that maps each word to an integer id.

In large datasets, there are too many words and most of them don't occur very frequently. One common approach we use to tackle this problem is to pick most common N words from the dataset. Therefore, we restrict the number of words.

Let's first compute the word frequencies in our dummy dataset. To compute frequencies, we use the [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) data structure.

In [5]:
from collections import Counter

freqs = Counter()
for tokens, _ in tokenized_data:
    freqs.update(tokens)

freqs

Counter({'obviously': 1,
         'written': 1,
         'for': 1,
         'the': 4,
         'stage': 1,
         'lightweight': 1,
         'but': 2,
         'worthwhile': 1,
         'how': 2,
         'can': 1,
         'you': 2,
         'go': 1,
         'wrong': 1,
         'with': 1,
         'ralph': 1,
         'richardson': 1,
         'olivier': 1,
         'and': 2,
         'merle': 1,
         'oberon': 1,
         'this': 4,
         'is': 5,
         'definitive': 1,
         'movie': 4,
         'version': 1,
         'of': 4,
         'hamlet': 1,
         'branagh': 1,
         'cuts': 1,
         'nothing': 1,
         'there': 1,
         'are': 1,
         'no': 1,
         'wasted': 1,
         'moments': 1,
         'smallville': 3,
         'episode': 3,
         'justice': 1,
         'best': 1,
         'it': 1,
         's': 1,
         'my': 1,
         'favorite': 1,
         'great': 1,
         'especially': 1,
         'music': 1,
         'etta': 1,

To create the dictionary, let's select the most common 20 words to create a vocabulary. In addition to the words that appear in our data, we need to have two special words:

- `<eos>` End of sequence symbol used for padding
- `<unk>` Words unknown in our vocabulary

In [6]:
vocab = {'<eos>': 0, '<unk>': 1}
for token, freq in freqs.most_common(20):
    vocab[token] = len(vocab)
vocab

{'<eos>': 0,
 '<unk>': 1,
 'is': 2,
 'the': 3,
 'this': 4,
 'movie': 5,
 'of': 6,
 'smallville': 7,
 'episode': 8,
 'but': 9,
 'how': 10,
 'you': 11,
 'and': 12,
 'bad': 13,
 'not': 14,
 'obviously': 15,
 'written': 16,
 'for': 17,
 'stage': 18,
 'lightweight': 19,
 'worthwhile': 20,
 'can': 21}

## 1.3 Creating the Dataset

Putting it all together, we can now create a dataset class. First, let's create index-label pairs:

In [7]:
indexed_data = []
for tokens, label in tokenized_data:
    indices = [vocab.get(token, vocab['<unk>']) for token in tokens]    
    # the token that is not in vocab get assigned <unk>
    indexed_data.append((indices, label))
    

for indices, label in indexed_data:
    print(indices, ' -> ', label)
    print()

[15, 16, 17, 3, 18, 19, 9, 20, 10, 21, 11, 1, 1, 1, 1, 1, 1, 12, 1, 1]  ->  1

[4, 2, 3, 1, 5, 1, 6, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1]  ->  1

[7, 8, 1, 2, 3, 1, 8, 6, 7, 1, 1, 1, 1, 8, 6, 7]  ->  1

[1, 5, 1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1]  ->  0

[1, 4, 5, 2, 1, 2, 1, 1, 1, 13, 1, 1, 1, 14, 1]  ->  0

[1, 1, 6, 1, 1, 14, 1, 1, 1, 10, 1, 1, 12, 1, 13, 4, 5, 2]  ->  0



<div class="alert alert-success"> 
    <h3>Task: Check Code</h3>
    <p>We now use the PyTorch dataset class we provided in <code>exercise_code/rnn/sentiment_dataset.py</code> file. Please also take a look at the code.</p>
 </div>
    


Dataset class also reverse sorts the sequences with respect to the lengths. Thanks to this sorting, we can reduce the total number of padded elements, which means that we have less computations for padded values.

In [8]:
from exercise_code.rnn.sentiment_dataset import SentimentDataset

combined_data = [
    (raw_text, tokens, indices, label)
    for (raw_text, label), (tokens, _), (indices, _)
    in zip(data, tokenized_data, indexed_data)
]

dataset = SentimentDataset(combined_data)

for elem in dataset:
    print(elem)
    print()

{'data': tensor([15, 16, 17,  3, 18, 19,  9, 20, 10, 21, 11,  1,  1,  1,  1,  1,  1, 12,
         1,  1]), 'label': tensor(1.)}

{'data': tensor([ 1,  5,  1,  3,  1,  1,  1,  1,  1,  4,  1,  1,  1, 11,  1,  1,  1,  1,
         1,  1]), 'label': tensor(0.)}

{'data': tensor([ 1,  1,  6,  1,  1, 14,  1,  1,  1, 10,  1,  1, 12,  1, 13,  4,  5,  2]), 'label': tensor(0.)}

{'data': tensor([4, 2, 3, 1, 5, 1, 6, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1]), 'label': tensor(1.)}

{'data': tensor([7, 8, 1, 2, 3, 1, 8, 6, 7, 1, 1, 1, 1, 8, 6, 7]), 'label': tensor(1.)}

{'data': tensor([ 1,  4,  5,  2,  1,  2,  1,  1,  1, 13,  1,  1,  1, 14,  1]), 'label': tensor(0.)}



## 1.4 Minibatching
Note that in the dataset we created, not all sequences have the same length. Therefore, we cannot minibatch the data trivially. This means we cannot use a `DataLoader` class easily.

<b>If you uncomment the following cell and run it, you will very likely get an error!</b>

In [9]:
#loader = DataLoader(dataset, batch_size=3)

#for batch in loader:
#     print(batch)

<div class="alert alert-success"> 
    <h3>Task: Check Code</h3>
    <p>To solve the problem, we need to pad the sequences with <code> < eos > </code> tokens that we indexed as zero. To integrate this approach into the Pytorch <a href="https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader" target="_blank">Dataloader</a> class, we will make use of the <code>collate_fn</code> argument. For more details, check out the <code>collate</code> function in <code>exercise_code/rnn/sentiment_dataset</code>. </p>
    <p> In addition, we use the <a href="https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html" target="_blank">pad_sequence</a> that pads shorter sequences with 0. </p>
 </div>

In [10]:
from torch.nn.utils.rnn import pad_sequence

def collate(batch):
    assert isinstance(batch, list)
    data = pad_sequence([b['data'] for b in batch])
    lengths = torch.tensor([len(b['data']) for b in batch])
    label = torch.stack([b['label'] for b in batch])
    return {
        'data': data,
        'label': label,
        'lengths': lengths
    }

loader = DataLoader(dataset, batch_size=3, collate_fn=collate)
for batch in loader:
    #print("hi")
    print('Data: \n', batch['data'])
    print('\nLabels: \n', batch['label'])
    print('\nSequence Lengths: \n', batch['lengths'])
    print('\n')

Data: 
 tensor([[15,  1,  1],
        [16,  5,  1],
        [17,  1,  6],
        [ 3,  3,  1],
        [18,  1,  1],
        [19,  1, 14],
        [ 9,  1,  1],
        [20,  1,  1],
        [10,  1,  1],
        [21,  4, 10],
        [11,  1,  1],
        [ 1,  1,  1],
        [ 1,  1, 12],
        [ 1, 11,  1],
        [ 1,  1, 13],
        [ 1,  1,  4],
        [ 1,  1,  5],
        [12,  1,  2],
        [ 1,  1,  0],
        [ 1,  1,  0]])

Labels: 
 tensor([1., 0., 0.])

Sequence Lengths: 
 tensor([20, 20, 18])


Data: 
 tensor([[ 4,  7,  1],
        [ 2,  8,  4],
        [ 3,  1,  5],
        [ 1,  2,  2],
        [ 5,  3,  1],
        [ 1,  1,  2],
        [ 6,  8,  1],
        [ 1,  6,  1],
        [ 1,  7,  1],
        [ 1,  1, 13],
        [ 1,  1,  1],
        [ 9,  1,  1],
        [ 1,  1,  1],
        [ 1,  8, 14],
        [ 1,  6,  1],
        [ 1,  7,  0],
        [ 1,  0,  0]])

Labels: 
 tensor([1., 1., 0.])

Sequence Lengths: 
 tensor([17, 16, 15])




We can see that these two batches have different length, this is how the reverse sort mentioned in `1.3 Creating the Dataset` benefits for less memory and less computation.

# 2. Embeddings

In the previous section, we explored how to convert text into a sequence of integers. In this form, sequences are still not ready to be inputs of RNNs you implemented in the optional notebook. Integer representation is some kind of one-hot encoding, while not the same since they are not equally weighted given only an integer. 

Moreover, it fails to express the semantic relations between words and the order of the words has no meaning. We would like a better representation form to keep semantic meaning of the word. For example, as shown in the following picture, the difference between man and woman and difference between king and queen should be close, since the difference is only the gender. If we use a vector for each word, the above relation can be expressed as $vec(\text{women})-vec(\text{man}) \approx vec(\text{queen}) - vec(\text{king})$. Usually we call such vector representations as embeddings.

<img src='https://developers.google.com/machine-learning/crash-course/images/linear-relationships.svg' width=80% height=80%/>

While one can use pre-trained embedding vectors such as [word2vec](https://arxiv.org/abs/1301.3781) or [GLoVe](https://nlp.stanford.edu/projects/glove/), in this exercise we use randomly initialized embedding vectors that will be trained from scratch together with our networks.

<div class="alert alert-info">

<h3> Task: Implement Embedding</h3>
 <p>In this part, you will implement a simple embedding layer. Embedding is a simple lookup table that stores a dense vector to represent each word in the vocabulary.</p> 

 <p>Your task is to implement the <code>Embedding</code> class in <code>exercise_code.rnn.rnn_nn</code> file. Once you are done, run the below cell to test your implementation. Note that we ensure eos embeddings to be zero by using the <code>padding_idx</code> argument.

 </div>

In [11]:
import torch.nn as nn

from exercise_code.rnn.rnn_nn import Embedding
from exercise_code.rnn.tests import embedding_output_test

#print(len(vocab))

i2dl_embedding = Embedding(len(vocab), 16, padding_idx=0)
pytorch_embedding = nn.Embedding(len(vocab), 16, padding_idx=0)

loader = DataLoader(dataset, batch_size=len(dataset), collate_fn=collate)
for batch in loader:
    x = batch['data']

embedding_output_test(i2dl_embedding, pytorch_embedding, x)


weight: torch.Size([22, 16])
inputs: torch.Size([20, 6])
embeddings: torch.Size([20, 6, 16])
Difference between outputs: 0.0
Test passed :)!


True

# 3. Conclusion

In this notebook, you learned how to prepare text data and how to create an embedding layer. In the next notebook, you will combine your Embedding and RNN implementations to create a sentiment analysis network!